In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import json
import urllib.request
import pickle
import datetime
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from scipy.stats import randint
from random import uniform

%matplotlib inline
fig = plt.figure(figsize=(20,5))

print(os.getcwd())

데이터셋 로드해서 예측 진행할 때마다 prediction 저장

In [ ]:
pred = {'na_pred' : na_pred,
        'mean_pred' : mean_pred,
        'median_pred' : median_pred,
        'linear_pred' : linear_pred,
        'time_pred' : time_pred,
        'spline_pred' : spline_pred}

pd.DataFrame(data = pred)

In [ ]:
def pred_graph(X_train, y_train, X_val, y_val, pred, sid1, sid2, sid3):
    
    # concat for sorting
    train = pd.concat([X_train, y_train], axis = 1)
    val = pd.concat([X_val, y_val], axis = 1)
    
    X = X_val.reset_index(drop = True)
    pred = pd.concat([X, pd.DataFrame(data = pred)], axis = 1)
    
    org_df = pd.concat([train, val])
    pred_df = pd.concat([train, pred])
    
    org_df.sort_values(by = ['store_id', 'year', 'month'], inplace = True)
    pred_df.sort_values(by = ['store_id', 'year', 'month'], inplace = True)
    
    org_df['date'] = org_df['year'].astype('str') + '-' + org_df['month'].astype('str')
    pred_df['date'] = pred_df['year'].astype('str') + '-' + pred_df['month'].astype('str')
    
    org_df = org_df[['store_id', 'date', 'amount']]
    pred_df = pred_df[['store_id', 'date', 'na_pred', 'mean_pred', 'median_pred', 'linear_pred', 'time_pred', 'spline_pred']]
    
    org_df.reset_index(inplace = True, drop = True)
    pred_df.reset_index(inplace = True, drop = True)
    
    
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.rcParams["figure.figsize"] = (20, 10)
    
    ## sid = store_id
    orgsub1 = org_df[org_df['store_id'] == sid1]
    predsub1 = pred_df[pred_df['store_id'] == sid1]
    
    orgsub2 = org_df[org_df['store_id'] == sid2]
    predsub2 = pred_df[pred_df['store_id'] == sid2]
    
    orgsub3 = org_df[org_df['store_id'] == sid3]
    predsub3 = pred_df[pred_df['store_id'] == sid3]

    plt.subplot(311)
    plt.title('Store ID : ' + str(sid1))
    plt.plot('date', 'amount', data = orgsub1, color = 'blue', linewidth = 1)
    plt.plot('date', 'na_pred', data = predsub1, color = 'red', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'mean_pred', data = predsub1, color = 'yellow', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'median_pred', data = predsub1, color = 'green', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'linear_pred', data = predsub1, color = 'magenta', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'time_pred', data = predsub1, color = 'red', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'spline_pred', data = predsub1, color = 'red', linestyle = 'dashed',linewidth = 1)

    plt.subplot(312)
    plt.title('Store ID : ' + str(sid2))
    plt.plot('date', 'amount', data = orgsub2, color = 'blue', linewidth = 1)
    plt.plot('date', 'na_pred', data = predsub2, color = 'red', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'mean_pred', data = predsub2, color = 'yellow', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'median_pred', data = predsub2, color = 'green', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'linear_pred', data = predsub2, color = 'magenta', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'time_pred', data = predsub2, color = 'red', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'spline_pred', data = predsub2, color = 'red', linestyle = 'dashed',linewidth = 1)

    plt.subplot(313)
    plt.title('Store ID : ' + str(sid3))
    plt.plot('date', 'amount', data = orgsub3, color = 'blue', linewidth = 1)
    plt.plot('date', 'na_pred', data = predsub3, color = 'red', linestyle = 'dashed', linewidth = 1)
    plt.plot('date', 'mean_pred', data = predsub3, color = 'yellow', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'median_pred', data = predsub3, color = 'green', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'linear_pred', data = predsub3, color = 'magenta', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'time_pred', data = predsub3, color = 'red', linestyle = 'dashed',linewidth = 1)
    plt.plot('date', 'spline_pred', data = predsub3, color = 'red', linestyle = 'dashed',linewidth = 1)

    plt.legend()
    plt.show()

pred_graph(X_train, y_train, X_val, y_val, pred, 10, 500, 1800)